# MergeAdjacentImages Notebook

This notebook contains a few examples of how to use the `MergeAdjacentImages` application from **TubeTK**. First, we will include Python's `os` package as well as `ITK` (Python wrapping). We also set the TubeTK build directory variable `TUBETK_BUILD_DIR`:

If ITK is not installed in your python environment, you need to define the environment variable `ITK_BUILD_DIR` that contains the path to where ITK was built.

We need to find the directory in which TubeTK was build. This is required to find the path to the testing data, and may be also required to find the TubeTK library paths if your python environment does not include it.
The environment variable `TubeTK_BUILD_DIR` needs to be defined.

In [1]:
import os
import sys
import numpy

In [2]:
import itk
from itk import TubeTK as ttk
from itkwidgets import view

In [3]:
import matplotlib.pyplot as plt
from matplotlib import cm

%matplotlib inline

Next, we load the first input image and show it's origin, spacing, etc.:

In [101]:
ImageType=itk.Image[itk.F,2]

input_image1 = "Data\ES0015_Large.mha"
reader0 = itk.ImageFileReader[ImageType].New(FileName=input_image1)
reader0.Update()

resampler0 = ttk.ResampleImage[itk.F,2].New(Input=reader0.GetOutput(), ResampleFactor=(0.5,0.5))
resampler0.Update()
im0 = resampler0.GetOutput()

print("Origin:")
print(im0.GetOrigin())
print("Spacing:")
print(im0.GetSpacing())

Origin:
itkPointD2 ([200, 200])
Spacing:
itkVectorD2 ([2, 2])


In [137]:
view(im0)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF2; proxy o…

Now let's split it into two overlaping images.

In [140]:
CropImageFilterType = ttk.CropImage[ImageType,ImageType]

crop1 = CropImageFilterType.New(Input=im0, Min=(0,0), Max=(200,200))
crop1.Update()
im1 = crop1.GetOutput()

In [141]:
view(im1)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF2; proxy o…

In [142]:
crop2 = CropImageFilterType.New(Input=im0, Min=(50,100), Max=(255,255))
crop2.Update()
im2 = crop2.GetOutput()

In [143]:
view(im2)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF2; proxy o…

Now let's merge the images without registration

In [144]:
MAIFilterType = ttk.MergeAdjacentImages[ImageType]

merge1 = MAIFilterType.New(Input1=im1, Input2=im2)
merge1.SetMaxIterations(0)
merge1.Update()
im12 = merge1.GetOutput()

In [145]:
view(im12)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF2; proxy o…

Now rotate and translate one image

In [150]:
TransformType = itk.AffineTransform[itk.D, 2]
trans = TransformType.New()
trans.Rotate2D( 3.1415926 * 0.02 )

ResampleFilterType = ttk.ResampleImage[itk.F, 2]
resample2 = ResampleFilterType.New(Input=im2, Transform=trans)
resample2.SetLoadTransform( True )
resample2.Update()
im2T = resample2.GetOutput()

merge2 = MAIFilterType.New(Input1=im1, Input2=im2T)
merge2.SetBlendUsingAverage( True )
merge2.SetMaxIterations(0)
merge2.Update()
im12T = merge2.GetOutput()

In [151]:
view(im12T)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF2; proxy o…

Now run merge with registration

In [152]:
merge3 = MAIFilterType.New(Input1=im1, Input2=im2T)
merge3.SetMaxIterations(1000)
merge3.SetExpectedRotation(0.1)
merge3.SetExpectedOffset(1)
merge3.SetSamplingRatio(0.5)
merge3.SetPadding((10,10))
merge3.Update()
im12TR = merge3.GetOutput()

In [153]:
view(im12TR)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF2; proxy o…